In [1]:
# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing")
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, "stowing-result")
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "success"]

In [4]:
RTYPE = "indy7"
DAT_DIR = '20210613-035257'
FILE_OPTION = "obj_1"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210613-035119'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 10
VISUALIZE = False
PLAY_RESULT = False
TIMEOUT_MOTION = 5
MAX_TIME = 100
SHOW_STATE = False


In [5]:

I_START = 20

### test

In [6]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
N_DAT = 10
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 5.7     4.5     24.1    6.5     -------
========== 1 / 10 (eta: 40.8 / 408.2 s) =========
------- 10.6    3.8     4.0     4.1     -------
========== 2 / 10 (eta: 63.3 / 316.4 s) =========
------- 4.0     3.8     4.1     4.0     -------
========== 3 / 10 (eta: 79.2 / 264.0 s) =========
------- 6.2     5.2     5.4     4.6     -------
========== 4 / 10 (eta: 100.7 / 251.7 s) =========
------- 3.4     4.5     4.3     7.7     -------
========== 5 / 10 (eta: 120.6 / 241.1 s) =========
------- 112.0   4.6     105.4   4.4     -------
========== 6 / 10 (eta: 347.1 / 578.4 s) =========
------- 3.6     3.7     29.1    4.3     -------
========== 7 / 10 (eta: 387.8 / 554.0 s) =========
------- 5.3     4.5     4.1     76.2    -------
========== 8 / 10 (eta: 477.8 / 597.3 s) =========
------- 79.1    17.1    6.0     5.6     -------
========== 9 / 10 (eta: 585.6 / 650.7 s) =========
------- 10.8    8.1     4.0     4.3     -------
========== 10 / 10 (eta: 612.8 / 612.8 s) =========


### load results

In [7]:
res_dict_all = defaultdict(lambda: defaultdict(list))
resfile_list = sorted(os.listdir(os.path.join(RESULTSET_PATH)))
for resfile in resfile_list:
    args = resfile[:-4].split("_")[1:]
    f_option = "_".join(args[:-2])
    fid, cname = args[-2:]
    resdat = load_pickle(os.path.join(RESULTSET_PATH, resfile))
    assert len(res_dict_all[f_option][cname]) == int(fid)
    res_dict_all[f_option][cname].append([resdat[head] for head in HEADS])

In [8]:
exp_list = ['obj_1', 'obj_3', 'obj_3_pole']
for ekey in exp_list:
    if ekey not in res_dict_all:
        continue
    res_dict = res_dict_all[ekey]
    print("="*30 + " {:<15} ".format(ekey) + "="*30)
    print("{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format("Checker", "Planning time", "Motion count", "MP trials", "Failed MP", "Failures", "Total"))
    for cname in CNAME_LIST:
        if cname in res_dict:
            vv = np.array(res_dict[cname])
            if len(vv)>0:
                succ_vec = vv[:,-1]
                fail_vec = np.logical_not(succ_vec)
                idc_succ = np.where(succ_vec)[0]
                print(
                    "{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format(cname,
                                    *(list(np.round(np.mean(vv[idc_succ, :-1], axis=0), 2)) \
                                      +[np.sum(fail_vec), len(vv)])))
    print(" ")

============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 17.04           6.65            4.42            0.23            4               30             
Tool           : 6.17            5.76            3.76            0.16            5               30             
ToolReach      : 4.36            5.46            3.54            0.12            4               30             
Full           : 5.29            5.32            3.16            0.0             5               30             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 30.37           8.73            5.2             0.07            5               20             
Tool           : 9.77            5.27            5.

In [9]:
RTYPE = "indy7"
DAT_DIR = '20210613-035257'
FILE_OPTION = "obj_3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210613-035119'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 10
VISUALIZE = False
PLAY_RESULT = False
TIMEOUT_MOTION = 5
MAX_TIME = 100
SHOW_STATE = False


### test

In [10]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
N_DAT = 10
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 5.5     4.6     332.1   104.7   -------
========== 1 / 10 (eta: 446.9 / 4469.5 s) =========
------- 5.2     45.8    4.2     18.5    -------
========== 2 / 10 (eta: 520.6 / 2603.2 s) =========
------- 3.8     5.6     22.5    4.5     -------
========== 3 / 10 (eta: 557.1 / 1857.0 s) =========
------- 11.4    139.7   104.6   103.8   -------
========== 4 / 10 (eta: 916.6 / 2291.4 s) =========
------- 85.5    104.2   4.2     12.8    -------
========== 5 / 10 (eta: 1123.3 / 2246.7 s) =========
------- 5.6     3.9     4.3     4.4     -------
========== 6 / 10 (eta: 1141.5 / 1902.5 s) =========
------- 9.0     8.8     4.4     11.1    -------
========== 7 / 10 (eta: 1174.8 / 1678.3 s) =========
------- 11.4    103.6   14.8    104.2   -------
========== 8 / 10 (eta: 1408.9 / 1761.1 s) =========
------- 34.7    5.2     4.3     104.4   -------
========== 9 / 10 (eta: 1557.5 / 1730.5 s) =========
------- 22.6    37.7    46.7    18.9    -------
========== 10 / 10 (eta: 1683.3 / 1683.3 s) ===

### load results

In [11]:
res_dict_all = defaultdict(lambda: defaultdict(list))
resfile_list = sorted(os.listdir(os.path.join(RESULTSET_PATH)))
for resfile in resfile_list:
    args = resfile[:-4].split("_")[1:]
    f_option = "_".join(args[:-2])
    fid, cname = args[-2:]
    resdat = load_pickle(os.path.join(RESULTSET_PATH, resfile))
    res_dict_all[f_option][cname].append([resdat[head] for head in HEADS])

In [12]:
exp_list = ['obj_1', 'obj_3', 'obj_3_pole']
for ekey in exp_list:
    if ekey not in res_dict_all:
        continue
    res_dict = res_dict_all[ekey]
    print("="*30 + " {:<15} ".format(ekey) + "="*30)
    print("{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format("Checker", "Planning time", "Motion count", "MP trials", "Failed MP", "Failures", "Total"))
    for cname in CNAME_LIST:
        if cname in res_dict:
            vv = np.array(res_dict[cname])
            if len(vv)>0:
                succ_vec = vv[:,-1]
                fail_vec = np.logical_not(succ_vec)
                idc_succ = np.where(succ_vec)[0]
                print(
                    "{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format(cname,
                                    *(list(np.round(np.mean(vv[idc_succ, :-1], axis=0), 2)) \
                                      +[np.sum(fail_vec), len(vv)])))
    print(" ")

============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 17.04           6.65            4.42            0.23            4               30             
Tool           : 6.17            5.76            3.76            0.16            5               30             
ToolReach      : 4.36            5.46            3.54            0.12            4               30             
Full           : 5.29            5.32            3.16            0.0             5               30             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 26.9            8.13            4.91            0.04            7               30             
Tool           : 11.57           5.95            7.

In [13]:
RTYPE = "indy7"
DAT_DIR = '20210613-035257'
FILE_OPTION = "obj_3_pole"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210613-035119'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 10
VISUALIZE = False
PLAY_RESULT = False
TIMEOUT_MOTION = 5
MAX_TIME = 100
SHOW_STATE = False


### test

In [14]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
N_DAT = 10
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 23.2    24.1    6.3     65.4    -------
========== 1 / 10 (eta: 119.0 / 1190.4 s) =========
------- 102.9   153.7   37.7    22.5    -------
========== 2 / 10 (eta: 435.8 / 2179.2 s) =========
------- 23.4    36.3    104.3   104.2   -------
========== 3 / 10 (eta: 704.1 / 2346.9 s) =========
------- 96.1    12.0    88.7    21.8    -------
========== 4 / 10 (eta: 922.6 / 2306.5 s) =========
------- 5.0     129.7   104.4   104.3   -------
========== 5 / 10 (eta: 1265.9 / 2531.8 s) =========
------- 14.5    4.4     4.6     15.0    -------
========== 6 / 10 (eta: 1304.4 / 2174.0 s) =========
------- 89.4    26.2    167.8   4.6     -------
========== 7 / 10 (eta: 1592.5 / 2275.0 s) =========
------- 66.7    4.7     5.1     12.2    -------
========== 8 / 10 (eta: 1681.2 / 2101.5 s) =========
------- 103.8   103.8   104.2   65.2    -------
========== 9 / 10 (eta: 2058.1 / 2286.8 s) =========
------- 41.1    5.3     12.7    5.2     -------
========== 10 / 10 (eta: 2122.3 / 2122.3 s) ===

### load results

In [15]:
res_dict_all = defaultdict(lambda: defaultdict(list))
resfile_list = sorted(os.listdir(os.path.join(RESULTSET_PATH)))
for resfile in resfile_list:
    args = resfile[:-4].split("_")[1:]
    f_option = "_".join(args[:-2])
    fid, cname = args[-2:]
    resdat = load_pickle(os.path.join(RESULTSET_PATH, resfile))
#     assert len(res_dict_all[f_option][cname]) == int(fid)
    res_dict_all[f_option][cname].append([resdat[head] for head in HEADS])

In [17]:
exp_list = ['obj_1', 'obj_3', 'obj_3_pole']
for ekey in exp_list:
    if ekey not in res_dict_all:
        continue
    res_dict = res_dict_all[ekey]
    print("="*30 + " {:<15} ".format(ekey) + "="*30)
    print("{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format("Checker", "Planning time", "Motion count", "MP trials", "Failed MP", "Failures", "Total"))
    for cname in CNAME_LIST:
        if cname in res_dict:
            vv = np.array(res_dict[cname])
            if len(vv)>0:
                succ_vec = vv[:,-1]
                fail_vec = np.logical_not(succ_vec)
                idc_succ = np.where(succ_vec)[0]
                print(
                    "{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format(cname,
                                    *(list(np.round(np.mean(vv[idc_succ, :-1], axis=0), 2)) \
                                      +[np.sum(fail_vec), len(vv)])))
    print(" ")

============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 17.04           6.65            4.42            0.23            4               30             
Tool           : 6.17            5.76            3.76            0.16            5               30             
ToolReach      : 4.36            5.46            3.54            0.12            4               30             
Full           : 5.29            5.32            3.16            0.0             5               30             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 26.9            8.13            4.91            0.04            7               30             
Tool           : 11.57           5.95            7.

## Res - panda
```

```

## Res - indy7
```
============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 18.58           6.56            4.06            0.11            2               20             
Tool           : 6.96            6.12            4.0             0.18            3               20             
ToolReach      : 2.79            5.44            3.67            0.17            2               20             
Full           : 2.22            5.25            3.12            0.0             4               20             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 30.37           8.73            5.2             0.07            5               20             
Tool           : 9.77            5.27            5.47            0.07            5               20             
ToolReach      : 7.24            6.0             7.42            0.17            8               20             
Full           : 6.2             5.53            4.53            0.0             5               20             
 
============================== obj_3_pole      ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 45.17           8.36            5.07            0.21            6               20             
Tool           : 16.48           5.71            7.65            0.53            3               20             
ToolReach      : 18.59           6.23            10.46           0.69            7               20             
Full           : 11.28           5.29            5.14            0.14            6               20             
```